### Especificaciones.

Utilizando ambos datasets realiza lo siguiete:
- Cargar el dataset
- Crea un conjunto de entrenamiento del 70% de las intancias y el conjunto de pruebas con el 30% (set random_state = 0)
- Utilizando el conjunto de entrenamiento crea un conjunto de validación con 3 pliegues

Utilizando el conjunto de validación realiza lo siguiente con cada pliegue:
- Utiliza Naive Bayes con distribución normal para entrenar y prueba el modelo en cada pliegue
    - Determina la exactitud de cada pliegue
    - Determina el promedio de exactitud

In [1]:
from Modulos import k_pliegues as kp
from Modulos import Naive_bayes as nv
from Modulos import Multinomial as mlt

In [2]:
k = 3
path = 'pliegues_email/'
y_test = kp.pliegues_validacion('./datasets/emails.csv', k, path, 0.7, True, 0)

Terminado los 3 pliegues


In [3]:
lista_precision = nv.inicializa_gaus(path)

Pliegue 1, terminado
Pliegue 2, terminado
Pliegue 3, terminado


In [4]:
exactitud = nv.kpliegues_vs_test(path, y_test.iloc[:1034])

Pliegue 1, terminado
Pliegue 2, terminado
Pliegue 3, terminado


In [5]:
exactitud

[0.9748549323017408,
 0.9661508704061895,
 0.988394584139265,
 0.5899419729206963,
 0.6141199226305609,
 0.6295938104448743]

In [6]:
nv.get_tableau()

+-----------+----------------+--------------------------------------------+---------------------------------------+
| Dataset   | Clasificador   |  Exactitud, promedio de los tres pliegues  |  Exactitud obtenida pliegues vs test  |
+===========+================+============================================+=======================================+
| e_mail    | Naive Bayes    |                  0.976467                  |               0.611219                |
+-----------+----------------+--------------------------------------------+---------------------------------------+


- Utiliza Multinomial con distribución normal para entrenar y prueba el modelo en cada pliegue
    - Determina la exactitud de cada pliegue
    - Determina el promedio de exactitud de los 3 pliegues

In [7]:
lista_multi = mlt.inicia_multi(path)

Pliegue 1, terminado
Pliegue 2, terminado
Pliegue 3, terminado


In [10]:
lista_multi

[0.7021276595744681,
 0.7224371373307543,
 0.7311411992263056,
 0.723404255319149,
 0.6199226305609284,
 0.6170212765957447]

In [8]:
lista_test_vs_pliegue = mlt.multinomial_vs_test(path, y_test.iloc[:1034])

Pliegue 1, terminado
Pliegue 2, terminado
Pliegue 3, terminado


In [11]:
lista_test_vs_pliegue

[0.7021276595744681,
 0.7224371373307543,
 0.7311411992263056,
 0.723404255319149,
 0.6199226305609284,
 0.6170212765957447]

In [9]:
mlt.get_tableau()

+-----------+----------------+--------------------------------------------+---------------------------------------+
| Dataset   | Clasificador   |  Exactitud, promedio de los tres pliegues  |  Exactitud obtenida pliegues vs test  |
+===========+================+============================================+=======================================+
| e_mail    | multinomial    |                  0.718569                  |               0.653449                |
+-----------+----------------+--------------------------------------------+---------------------------------------+
